In [52]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
from packaging import version
import plotly.express as px
import plotly.io as pio
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as pyo
pio.renderers.default = 'notebook'
pyo.init_notebook_mode(connected=True)
from sklearn.preprocessing import StandardScaler, RobustScaler, OneHotEncoder, OrdinalEncoder, PowerTransformer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import f1_score

warnings.filterwarnings('ignore')

In [9]:
path = '../dataset/spend_analysis_dataset.csv'

In [57]:
df = pd.read_csv(path,parse_dates=["PurchaseDate"])

In [58]:
def all_des(df: pd.DataFrame):
    print(f'shape : {df.shape[0]} x {df.shape[1]}')
    print(f'{"           Describing"}')
    print(f'{"Col Names":<30} {"Unique":<30} {"Type":<30}')
    print('-'*60)

    for col in df.keys():
        print(f'{col:<30} {df[col].nunique():<30} {str(df[col].dtypes):<30}')

    print(f'{"           Missing"}')
    print(f'{"Col Names":<30} {"Counts":<30} ')
    print('-'*60)
    
    for col in df.keys():
        print(f'{col:<30} {df[col].isnull().sum():<30} ')

In [59]:
all_des(df)

shape : 500 x 9
           Describing
Col Names                      Unique                         Type                          
------------------------------------------------------------
TransactionID                  500                            object                        
ItemName                       10                             object                        
Category                       6                              object                        
Quantity                       20                             int64                         
UnitPrice                      485                            float64                       
TotalCost                      499                            float64                       
PurchaseDate                   266                            datetime64[ns]                
Supplier                       5                              object                        
Buyer                          20                             ob

In [60]:
df.describe().T

,count,mean,min,25%,50%,75%,max,std
Quantity,500.0,9.64,1.0,4.0,10.0,15.0,20.0,6.168834
UnitPrice,500.0,854.2451,1.06,14.38,143.595,287.925,9909.24,2132.890832
TotalCost,500.0,2481.16106,1.56,145.9275,918.56,3123.5675,18494.6,3527.35316
PurchaseDate,500,2024-06-23 19:40:48,2024-01-01 00:00:00,2024-03-22 18:00:00,2024-06-14 12:00:00,2024-09-18 00:00:00,2024-12-30 00:00:00,NaN


In [61]:
df.sample(5)

,TransactionID,ItemName,Category,Quantity,UnitPrice,TotalCost,PurchaseDate,Supplier,Buyer
266,TXN267,Laptop Bag,Accessories,6,26.64,159.84,2024-10-27,QuickDeliver Ltd.,Kelly Joseph
22,TXN023,Whiteboard,Furniture,16,150.60,2409.60,2024-08-30,FurniWorks Ltd.,Dawn Padilla
194,TXN195,Annual Software License,Software,1,8614.00,8614.00,2024-12-03,OfficeSupplies Co.,Kayla Hanson
449,TXN450,Annual Software License,Software,1,5047.01,5047.01,2024-09-21,OfficeSupplies Co.,Dawn Padilla
297,TXN298,Annual Software License,Software,1,5489.81,5489.81,2024-11-02,TechMart Inc.,Todd James


In [70]:
df["Month"] = df["PurchaseDate"].dt.to_period("M").astype(str)
df["Year"] = df["PurchaseDate"].dt.to_period("Y").astype(str)
df["Day"] = df["PurchaseDate"].dt.to_period("D").astype(str)

## EDA

In [71]:
def make_box_hist(df: pd.DataFrame):
    fig = make_subplots(rows=2, cols=1)

    f1 = go.Box(x=df, name=df.name , boxmean=True)
    f2 = go.Histogram(x=df,name=df.name)

    fig.add_trace(f1,row=1,col=1)
    fig.add_trace(f2,row=2,col=1)

    fig.update_layout(width=1000,height=600,title_text=df.name)
    fig.show()

In [72]:
num_col = [col for col in df.keys() if df[col].dtypes in ['int64', 'float64']]
num_col

['Quantity', 'UnitPrice', 'TotalCost']

In [73]:
make_box_hist(df['Quantity'])

In [74]:
make_box_hist(df['UnitPrice'])

In [75]:
make_box_hist(df['TotalCost'])

In [76]:
def make_scatter(df: pd.DataFrame, col1 : str, col2 : str, hue: str):
    fig = px.scatter(df, x=col1, y=col2, color=hue)
    fig.update_layout(width=1000,height=600,title_text= col1 +" vs "+ col2)
    fig.show()

In [77]:
make_scatter(df,'Quantity','UnitPrice','Category')

In [80]:
def top_10(df: pd.DataFrame, group : str, col1 :str):
    top_items = df.groupby(group)[col1].sum().nlargest(10).reset_index()

    fig = px.bar(top_items, x=group, y=col1, title="Top 10 สินค้าที่ใช้จ่ายมากที่สุด", text_auto=True)
    fig.show()

In [81]:
top_10(df, 'ItemName','TotalCost')

In [86]:
def compare_category(df: pd.DataFrame, group: str, col1: str):
    cat_cost = df.groupby(group)[col1].sum().reset_index()

    fig = px.pie(cat_cost, values=col1, names=group, title="สัดส่วนค่าใช้จ่ายตามประเภทสินค้า")
    fig.show()

In [87]:
compare_category(df, "Category", "TotalCost")

In [90]:
def cost_per(df, group, col):
    cost = df.groupby(group)[col].sum().reset_index()

    fig = px.line(cost, x=group, y=col, title="แนวโน้มค่าใช้จ่ายรายเดือน")
    fig.show()

In [91]:
cost_per(df, "Month", "TotalCost")

In [94]:
cost_per(df, "Day", "TotalCost")

In [95]:
top_10(df, 'Supplier','TotalCost')

In [96]:
fig = px.box(df, x="Category", y="UnitPrice", title="การกระจายราคาต่อหน่วยในแต่ละประเภทสินค้า")
fig.show()

In [97]:
outliers = df[df["TotalCost"] > df["TotalCost"].quantile(0.95)]

fig = px.scatter(outliers, x="ItemName", y="TotalCost", color="Supplier", title="Outliers - รายการที่ต้นทุนสูงผิดปกติ")
fig.show()

In [98]:
reordered = df.groupby("ItemName")["Buyer"].nunique().reset_index()
reordered = reordered.sort_values("Buyer", ascending=False).head(10)

fig = px.bar(reordered, x="ItemName", y="Buyer", title="Top 10 สินค้าที่ผู้ซื้อหลายคนซื้อซ้ำ", text_auto=True)
fig.show()

In [99]:
# นับจำนวนครั้งที่ Buyer ซื้อ Item เดิม
repeat = df.groupby(["Buyer", "ItemName"]).size().reset_index(name='Times')

# เอาเฉพาะที่ซื้อซ้ำ
repeat = repeat[repeat["Times"] > 1].sort_values("Times", ascending=False)

fig = px.bar(repeat.head(10), x="ItemName", y="Times", color="Buyer",
             title="Top สินค้าที่ผู้ซื้อรายเดิมสั่งซ้ำมากที่สุด", text_auto=True)
fig.show()

In [100]:
df_sorted = df.sort_values(["Buyer", "ItemName", "PurchaseDate"])
df_sorted["PrevPurchase"] = df_sorted.groupby(["Buyer", "ItemName"])["PurchaseDate"].shift(1)
df_sorted["ReorderGapDays"] = (df_sorted["PurchaseDate"] - df_sorted["PrevPurchase"]).dt.days

reorder_gap = df_sorted[df_sorted["ReorderGapDays"].notna()]
avg_gap = reorder_gap.groupby("ItemName")["ReorderGapDays"].mean().reset_index().sort_values("ReorderGapDays")

fig = px.bar(avg_gap.head(10), x="ItemName", y="ReorderGapDays",
             title="Top 10 สินค้าที่มีรอบการสั่งซ้ำเฉลี่ยน้อย (ซื้อบ่อย)", text_auto=True)
fig.show()

In [101]:
monthly_reorder = df.groupby(["ItemName", "Month"])["TransactionID"].count().reset_index()
active_months = monthly_reorder.groupby("ItemName")["Month"].nunique().reset_index(name="ActiveMonths")

fig = px.bar(active_months.sort_values("ActiveMonths", ascending=False).head(10),
             x="ItemName", y="ActiveMonths",
             title="Top สินค้าที่ถูกสั่งซื้อต่อเนื่องหลายเดือน", text_auto=True)
fig.show()

## Recomander System

In [102]:
!pip install mlxtend

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------- -------------------------------- 0.3/1.4 MB ? eta -:--:--
   --------------- ------------------------ 0.5/1.4 MB 1.7 MB/s eta 0:00:01
   ------------------------------- -------- 1.0/1.4 MB 1.8 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 1.8 MB/s eta 0:00:00


In [112]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

In [159]:
basket = df.groupby("TransactionID")["ItemName"].apply(list).tolist()

# แปลงเป็น binary matrix
te = TransactionEncoder()
te_ary = te.fit(basket).transform(basket)
basket_df = pd.DataFrame(te_ary, columns=te.columns_)

In [164]:
# หาชุดสินค้าที่พบบ่อย
frequent_itemsets = apriori(basket_df, min_support=0.002, use_colnames=True)

# หากฎความสัมพันธ์
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.1)

# แสดงตัวอย่าง
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].head()

,antecedents,consequents,support,confidence,lift


In [165]:
rules['antecedents'] = rules['antecedents'].apply(lambda x: ', '.join(list(x)))
rules['consequents'] = rules['consequents'].apply(lambda x: ', '.join(list(x)))

def recommend_items(item_name, rules_df, top_n=5):
    matched = rules_df[rules_df['antecedents'].str.contains(item_name, case=False)]
    matched = matched.sort_values(by="lift", ascending=False).head(top_n)

    if matched.empty:
        print(f"❌ ไม่มีคำแนะนำที่เกี่ยวข้องกับ: {item_name}")
    else:
        print(f"🔮 คำแนะนำเมื่อซื้อ '{item_name}':\n")
        print(matched[['antecedents', 'consequents', 'support', 'confidence', 'lift']])

        # กราฟ
        fig = px.bar(matched, x='consequents', y='lift', text='confidence',
                     title=f"สินค้าที่ควรแนะนำเมื่อซื้อ '{item_name}'", color='lift')
        fig.show()

In [166]:
recommend_items("Printer Ink", rules)

❌ ไม่มีคำแนะนำที่เกี่ยวข้องกับ: Printer Ink


In [167]:
df.groupby("TransactionID")["ItemName"].nunique().describe()

count    500.0
mean       1.0
std        0.0
min        1.0
25%        1.0
50%        1.0
75%        1.0
max        1.0
Name: ItemName, dtype: float64

In [168]:
df["ItemName"].value_counts().head(10)

ItemName
Stapler                    60
Laptop Bag                 56
Laptop                     56
Desk Chair                 53
Whiteboard                 49
Notepad                    48
Annual Software License    46
Printer                    45
Monitor                    45
Printer Ink                42
Name: count, dtype: int64